In [4]:
-- setup some useful stuff
require 'nn'

-- help function to print in green
function cprint(str) print(sys.COLORS.green..str..'\27[0m') end

-- tests
precision = 1e-5
tester = torch.Tester()
function runTest(test)
    tester:add(test)
    tester:run()
    tester = torch.Tester()
end

-- setting the random generator seed
torch.manualSeed(42)

##Lookup Table

In [ ]:
lookupTable = function(vocabSize, embeddingSize)
    
end

##Linear Module

In [5]:
linearModule = function(inputSize, outputSize)
    
    -- module to build
    local this = {}
    
    -- standard deviation for initialization
    local stdv =  1./math.sqrt(outputSize)
    
    -- weight matrix
    this.weight = torch.Tensor(outputSize, inputSize):uniform(-stdv, stdv)
    
    -- bias vector
    this.bias = torch.Tensor(outputSize, 1):uniform(-stdv, stdv)
    
    -- forward operation
    this.forward = function(input)
        -- quirk to make it work with vectors and matrices
        input = (input:dim() == 1) and input:reshape(input:size(1), 1) or input
        -- multiply the input and weight matrix
        local output = this.weight * input
        -- add the expanded bias vector and return
        return output + this.bias:expand(output:size())
    end

    return this
end

runTest(function()
    print()
    local layer = linearModule(2,3)
    layer.weight:fill(2)
    layer.bias:fill(1)
    cprint('LinearModule weight is now:')
    print(layer.weight)
    cprint('LinearModule bias is now:')
    print(layer.bias)

    local inputVector = torch.Tensor(2):fill(-1)
    cprint('Testing input vector')
    print(inputVector)
    local output = layer.forward(inputVector)
    cprint('Output is:')
    print(output)
    local expected = torch.Tensor(3,1):fill(-3)
    tester:assertTensorEq(expected, output, precision)

    local inputMatrix = torch.Tensor(2, 3):fill(2)
    cprint('Testing input matrix')
    print(inputMatrix)
    local output = layer.forward(inputMatrix)
    cprint('Output is:')
    print(output)
    local expected = torch.Tensor(3,3):fill(9)
    tester:assertTensorEq(expected, output, precision)
end)

Running 1 tests	
|  ==> unknown
LinearModule weight is now:	
 2  2
 2  2
 2  2
[torch.DoubleTensor of dimension 3x2]

LinearModule bias is now:	
 1
 1
 1
[torch.DoubleTensor of dimension 3x1]

Testing input vector	
-1
-1
[torch.DoubleTensor of dimension 2]

Output is:	


-3
-3
-3
[torch.DoubleTensor of dimension 3x1]

Testing input matrix	
 2  2  2
 2  2  2
[torch.DoubleTensor of dimension 2x3]

Output is:	
 9  9  9
 9  9  9
 9  9  9
[torch.DoubleTensor of dimension 3x3]



_  ==> Done 

Completed 2 asserts in 1 tests with 0 errors	

--------------------------------------------------------------------------------	


##Sigmoid

In [ ]:
sigmoid = function(input)
end

## LogSoftMax
\begin{equation*}
   LogSoftMax(x_i) = -\ln \Bigl(\frac{1}{e^{x_i}} \sum_j e^{x_j}\Bigr)
\end{equation*}

In [47]:
logSoftMax = function(input) 
    
    -- quirk to make it work with vectors and matrices
    input = (input:dim() == 1) and input:reshape(input:size(1), 1) or input
    
    -- calculate sum of e^x_i and expand it to the right size
    local sumOfExp = torch.exp(input):sum(2):expand(input:size())
    
    -- calculate the rest of the formula and return
    return torch.exp(input):pow(-1):cmul(sumOfExp):log():mul(-1)
end


runTest(function()
    print()
    input = torch.range(1,9):reshape(3,3)
    cprint('Input matrix is:')
    print(input)
    local lms = nn.LogSoftMax()
    local expected = lms:forward(input)
    cprint('Output from nn.LogSoftMax is:')
    print(expected)
    local output = logSoftMax(input)
    cprint('Output from logSoftMax is:')
    print(output)
    tester:assertTensorEq(expected, output, 1e-5)
end)

Running 1 tests	
|  ==> unknown
Input matrix is:	
 1  2  3
 4  5  6
 7  8  9
[torch.DoubleTensor of dimension 3x3]



Output from nn.LogSoftMax is:	
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
[torch.DoubleTensor of dimension 3x3]

Output from logSoftMax is:	
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
[torch.DoubleTensor of dimension 3x3]

_  ==> Done 

Completed 1 asserts in 1 tests with 0 errors	

--------------------------------------------------------------------------------	


In [8]:
-- network parameters
local embeddingSize = 2
local contextLength = 3
local vocabSize = 5
local hiddenSize = 6
local stdv = 1

-- IndexToEmbedding
local lookupTable = linearModule(vocabSize, embeddingSize)
cprint('LookupTable of '..vocabSize..' words, each is a vector of size '..embeddingSize)
print(lookupTable.weight)
print(lookupTable.bias)

-- ContextToHidden
local contextToHidden = linearModule(contextLength * embeddingSize, hiddenSize)
cprint('Context-To-Hidden matrix is:')
print(contextToHidden.weight)
print(contextToHidden.bias)

local hiddenToEmbedding = torch.FloatTensor(embeddingSize, hiddenSize):uniform(-stdv, stdv)

cprint('Hidden-To-Embedding matrix is:')
print(hiddenToEmbedding)

local embeddingToVocabulary = torch.FloatTensor(vocabSize, embeddingSize):uniform(-stdv, stdv)

cprint('Embedding-To-Vocabulary matrix is:')
print(embeddingToVocabulary)

function softMax(matrix)
   -- -log(sum(exp(matrix)) * 1/exp(matrix))
   return torch.mul(torch.exp(matrix):pow(-1), torch.sum(torch.exp(matrix), 1)[1]):log():mul(-1)
end

-- Forward
--[[
local oneHot = torch.FloatTensor():eye(vocabSize)
cprint('1-Hot representation of second word is:')
print(oneHot[2])

local secondWord = torch.mv(lookupTable, oneHot[2])
cprint('Vector representation of second word is:')
print(secondWord)
--]]
local oneHotIndices = torch.LongTensor{1,3,5}
cprint('Context will be built of words at indices...')
print(oneHotIndices)

local contextMatrix = lookupTable:index(2, oneHotIndices)
cprint('...which corresponds to the following matrix:')
print(oneHotContext)
--[[
local contextMatrix = torch.mm(lookupTable, oneHotContext)
cprint('The corresponding matrix of embeddings are:')
print(contextMatrix)
--]]
local contextVector = torch.reshape(contextMatrix, contextMatrix:nElement())
cprint('...which reshaped as a vector is:')
print(contextVector)

local output = contextToHidden * contextVector
print(output)

output = torch.mv(hiddenToEmbedding, output)
print(output)

output = torch.mv(embeddingToVocabulary, output)
print(output)

output = softMax(output)
print(output)


LookupTable of 5 words, each is a vector of size 2	
-0.6836  0.2986  0.0171  0.4377 -0.3868
-0.2140  0.2053 -0.5711 -0.4605  0.6230
[torch.DoubleTensor of dimension 2x5]

 0.2700
-0.1449
[torch.DoubleTensor of dimension 2x1]

Context-To-Hidden matrix is:	
-0.0925  0.0145  0.3566  0.2757 -0.2960  0.1435
-0.1298  0.1921 -0.3156 -0.2375  0.3468  0.0338
 0.3081  0.1599 -0.1976 -0.2216  0.1306 -0.2654
 0.2590  0.3937  0.0451  0.0136  0.0242 -0.1953
-0.2108  0.4052 -0.3322  0.3800  0.3243  0.0476
 0.3269  0.3124  0.1087 -0.2542 -0.1314 -0.1806
[torch.DoubleTensor of dimension 6x6]

-0.1231
 0.1636
 0.1845
 0.2830
 0.3242
 0.2909
[torch.DoubleTensor of dimension 6x1]

Hidden-To-Embedding matrix is:	
 0.7742 -0.1910  0.5598  0.7755  0.2841  0.7019
-0.8317  0.8713 -0.6767  0.5707  0.7971  0.3380
[torch.FloatTensor of dimension 2x6]

Embedding-To-Vocabulary matrix is:	
 0.2129  0.1614
-0.9816 -0.2554
-0.7971  0.8803
 0.3270  0.9473
-0.9899 -0.4322
[torch.FloatTensor of dimension 5x2]

Context wi

[string "-- network parameters..."]:49: attempt to call method 'index' (a nil value)
stack traceback:
	[string "-- network parameters..."]:49: in main chunk
	[C]: in function 'xpcall'
	/home/fiskio/torch/install/share/lua/5.1/itorch/main.lua:174: in function </home/fiskio/torch/install/share/lua/5.1/itorch/main.lua:140>
	/home/fiskio/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/home/fiskio/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/home/fiskio/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/home/fiskio/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/fiskio/torch/install/share/lua/5.1/itorch/main.lua:341: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406170: 